Voici le projet d'OCVX du groupe de Hadrien Berthier, Axel Petit et Franck Thang
Ce projet est rélalisé en python 3, et utilisera les bibliothèques scipy numpy et panda
Ce document fait objet de rapport, en plus du code il comportera des explications et des annotations.

In [7]:
import numpy as np
import scipy as sc
import pandas as pd

In [1]:
def partial(f, x, i=0, dx=1e-6):
    """Computes i-th partial derivative of f at point x.
    
    Args:
        f: objective function.
        x: point at which partial derivative is computed.
        i: coordinate along which derivative is computed.
        dx: slack for finite difference.
        
    Output:
        (float)

    """
    x = x.reshape(1, -1)
    h = np.zeros(x.shape)
    h[0, i] = dx
    return (f(x + h) - f(x - h)) / (2*dx)

In [3]:
def gradient(f, x, dx=1e-6):
    """Computes gradient of f at point x.
    
    Args:
        f: objective function.
        x: point at which gradient is computed.
        dx: slack for finite difference of partial derivatives.
        
    Output:
        (ndarray) of size domain of f.
        
    """
    x = x.reshape(1, -1)
    dim = x.shape[1]
    return np.array([partial(f, x, i, dx) for i in range(dim)]).reshape(1, -1)

In [4]:
class GD():
    """Gradient Descent Object.
    
    Implements gradient descent aiming to compute optimal objective 
    value of convex functions and local optimal ones of none 
    convex functions.
    
    """    
    def __init__(self, d_dir=gradient, 
                 rate=(lambda x, y, z, grad: 0.01), 
                 decay=(lambda x: np.linalg.norm(x)), 
                 tol=1e-6, max_iter=1000, grad=gradient):
        """        
        Instantiates a GD object.
    
        Attributes:
        d_dir: function computing descent direction.
        rate: function computing learning rate ; takes in
              - f (function): objective function
              - x (ndarray): current iterate
              - dir_x (ndarray): output of a descent direction function
              - grad (ndarray): gradient function.
        decay: function computing decay.
        tol: slack tolerance.
        max_iter: upper bound on number of iterations.
    
        """
        self.d_dir = d_dir
        self.rate = rate
        self.decay = decay
        self.tol = tol
        self.max_iter = max_iter
        self.grad = gradient
    
    def __call__(self, x, f, rate=None):
        """Calling gradient descent object with specific starting point and optimal function.
        
        Args:
            x: initial starting point for descent.
            f: objective function of optimisation problem.
        
        Output:
            (float) sub-optimal value up to tolerance if execution is proper.
            (ndarray) list of gradient descent iterates.
            
        """
        x = x.reshape(1, -1)
        n_iter = 0
        dir_x = -self.d_dir(f, x, self.tol)
        if (rate == None):
            delta_x = self.rate(f, x, dir_x, n_iter) * dir_x
        else:
            delta_x = rate * dir_x
        iters, iters_dir = x, delta_x
        grad_f_x = self.grad(f, x)
        decay = self.decay(grad_f_x)
        while decay > self.tol and n_iter < self.max_iter:
            ## Deciding on direction
            dir_x = -self.d_dir(f, x, self.tol)
            if (rate == None):
                delta_x = self.rate(f, x, dir_x, n_iter) * dir_x
            else:
                delta_x = rate * dir_x            
            ## Updating iterate
            x = x + delta_x
            ## Storing on-going data
            iters_dir = np.vstack([iters_dir, delta_x])
            iters = np.vstack([iters, x])
            ## Computing decay
            grad_f_x = self.grad(f, x)
            decay = self.decay(grad_f_x)
            ## Updating iteration number
            n_iter += 1
        msg = " Iteration nu. = {}\n approx. = {}\n ob value = {}\n and decay = {}."
        print(msg.format(n_iter, x.flatten(), f(x), decay))
        if decay > self.tol:
            warnings.warn("Decay didn't get under tolerance rate.", RuntimeWarning)
        return (x, iters, iters_dir, n_iter)

In [12]:
DG_classic = GD()
op_pt, iters, iters_dir, n_iter = DG_classic(np.array([10]), lambda x: 3*x[0]**2 + 2*x[0] + 1)

 Iteration nu. = 290
 approx. = [-0.33333317]
 ob value = [0.66666667]
 and decay = 9.987566329527908e-07.
